In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



# Load the dataset
df = pd.read_csv('indian_rda_based_diet_recommendation_system.csv')  # Replace with the actual file name

# Clean the VegNovVeg column to remove any potential hidden characters or spaces
df['VegNovVeg'] = df['VegNovVeg'].astype(str).str.strip()

# Function to calculate BMI and categorize it
def calculate_bmi(weight, height):
    bmi = weight / ((height/100) ** 2)
    if bmi < 18.5:
        bmi_category = "Underweight"
    elif 18.5 <= bmi < 24.9:
        bmi_category = "Normal weight"
    elif 25 <= bmi < 29.9:
        bmi_category = "Overweight"
    else:
        bmi_category = "Obese"
    return bmi, bmi_category

# Function to calculate TDEE based on activity level
def calculate_tdee(bmr, activity_level):
    if activity_level == 'sedentary':
        tdee = bmr * 1.2
    elif activity_level == 'light':
        tdee = bmr * 1.375
    elif activity_level == 'moderate':
        tdee = bmr * 1.55
    elif activity_level == 'active':
        tdee = bmr * 1.725
    elif activity_level == 'very_active':
        tdee = bmr * 1.9
    else:
        tdee = bmr  # Default to sedentary if activity level is unknown
    return tdee

# Function to calculate BMR based on the Mifflin-St Jeor equation
def calculate_bmr(weight, height, age, gender):
    if gender.lower() == 'male':
        bmr = 10 * weight + 6.25 * height - 5 * age + 5
    elif gender.lower() == 'female':
        bmr = 10 * weight + 6.25 * height - 5 * age - 161
    else:
        bmr = 0  # Invalid gender
    return bmr

# Function to calculate calorie requirements based on weight loss plan
def calculate_calories_for_plan(tdee, weight_loss_plan):
    if weight_loss_plan == 'Maintain weight':
        return tdee
    elif weight_loss_plan == 'Lose Mild Weight':
        return tdee - 250  # Reduce by 250 calories for mild weight loss
    elif weight_loss_plan == 'Lose Basic Weight':
        return tdee - 500  # Reduce by 500 calories for basic weight loss
    elif weight_loss_plan == 'Lose Extreme Weight':
        return tdee - 750  # Reduce by 750 calories for extreme weight loss
    else:
        print("Invalid weight loss plan")
        return tdee

# Function to get breakfast items based on dietary preference and calorie target
def get_breakfast_items(dietary_preference, calorie_target):
    breakfast_df = df[df['Breakfast'] == 1]
    if dietary_preference.lower() == 'veg':
        filtered_df = breakfast_df[breakfast_df['VegNovVeg'] == '0']  # Veg items
    elif dietary_preference.lower() == 'non-veg':
        filtered_df = breakfast_df[breakfast_df['VegNovVeg'] == '1']  # Non-Veg items
    else:
        print("Invalid dietary preference for breakfast.")
        return [], 0
    filtered_df = filtered_df.sort_values(by='Calories', ascending=False)
    selected_food_items = []
    total_calories = 0
    for index, row in filtered_df.iterrows():
        if total_calories + row['Calories'] <= calorie_target:
            selected_food_items.append(row['Food_items'])
            total_calories += row['Calories']
        if total_calories >= calorie_target:
            break
    return selected_food_items, total_calories

# Function to get lunch items based on dietary preference and calorie target
def get_lunch_items(dietary_preference, calorie_target):
    lunch_df = df[df['Lunch'] == 1]
    if dietary_preference.lower() == 'veg':
        filtered_df = lunch_df[lunch_df['VegNovVeg'] == '0']  # Veg items
    elif dietary_preference.lower() == 'non-veg':
        filtered_df = lunch_df[lunch_df['VegNovVeg'] == '1']  # Non-Veg items
    else:
        print("Invalid dietary preference for lunch.")
        return [], 0
    filtered_df = filtered_df.sort_values(by='Calories', ascending=False)
    selected_food_items = []
    total_calories = 0
    for index, row in filtered_df.iterrows():
        if total_calories + row['Calories'] <= calorie_target:
            selected_food_items.append(row['Food_items'])
            total_calories += row['Calories']
        if total_calories >= calorie_target:
            break
    return selected_food_items, total_calories

# Function to get dinner items based on dietary preference and calorie target
def get_dinner_items(dietary_preference, calorie_target):
    dinner_df = df[df['Dinner'] == 1]
    if dietary_preference.lower() == 'veg':
        filtered_df = dinner_df[dinner_df['VegNovVeg'] == '0']  # Veg items
    elif dietary_preference.lower() == 'non-veg':
        filtered_df = dinner_df[dinner_df['VegNovVeg'] == '1']  # Non-Veg items
    else:
        print("Invalid dietary preference for dinner.")
        return [], 0
    filtered_df = filtered_df.sort_values(by='Calories', ascending=False)
    selected_food_items = []
    total_calories = 0
    for index, row in filtered_df.iterrows():
        if total_calories + row['Calories'] <= calorie_target:
            selected_food_items.append(row['Food_items'])
            total_calories += row['Calories']
        if total_calories >= calorie_target:
            break
    return selected_food_items, total_calories

# Main function to combine the meals and calculate calorie breakdown
def get_meal_plan(dietary_preference, weight, height, age, gender, activity_level, weight_loss_plan, total_calories):
    # Calculate BMI and BMI category
    bmi, bmi_category = calculate_bmi(weight, height)
    print(f"BMI: {bmi:.2f} ({bmi_category})")
    
    # Calculate BMR
    bmr = calculate_bmr(weight, height, age, gender)
    print(f"BMR: {bmr:.2f} kcal/day")
    
    # Calculate TDEE
    tdee = calculate_tdee(bmr, activity_level)
    print(f"TDEE (Total Daily Energy Expenditure): {tdee:.2f} kcal/day")
    
    # Calculate calorie requirements based on weight loss plan
    calorie_target_for_plan = calculate_calories_for_plan(tdee, weight_loss_plan)
    print(f"Calorie Requirement for {weight_loss_plan} Plan: {calorie_target_for_plan:.2f} kcal/day")

    # Calculate meal-specific calorie targets based on the total calorie input
    breakfast_calories = calorie_target_for_plan * 0.25  # 25% for breakfast
    lunch_calories = calorie_target_for_plan * 0.40    # 40% for lunch
    dinner_calories = calorie_target_for_plan * 0.35   # 35% for dinner

    # Get food items for each meal
    breakfast_items, breakfast_total = get_breakfast_items(dietary_preference, breakfast_calories)
    lunch_items, lunch_total = get_lunch_items(dietary_preference, lunch_calories)
    dinner_items, dinner_total = get_dinner_items(dietary_preference, dinner_calories)

    # Display the selected food items for each meal and the total calories for the day
    print("\nBreakfast: ")
    for item in breakfast_items:
        print(f"- {item}")
    print(f"Total Breakfast Calories: {breakfast_total}")

    print("\nLunch: ")
    for item in lunch_items:
        print(f"- {item}")
    print(f"Total Lunch Calories: {lunch_total}")

    print("\nDinner: ")
    for item in dinner_items:
        print(f"- {item}")
    print(f"Total Dinner Calories: {dinner_total}")

    total_day_calories = breakfast_total + lunch_total + dinner_total
    print(f"\nTotal Calories for the Day: {total_day_calories}, please add light snacks to your meals to meet your target Calorie requirements")
    return breakfast_items, lunch_items, dinner_items



# Bio Impedance Prediction using Random Forest
# Load the BMI dataset
df_bio = pd.read_csv('Body Mass Index.csv')

# Preprocess the data
df_bio['Height'] = df_bio['Height'] / 100  # Convert height to meters
df_bio['Gender'] = df_bio['Gender'].map({'M': 0, 'F': 1})

# Features (X) and Target (y)
X_bio = df_bio[['Age', 'Height', 'Weight', 'Gender']]
y_bio = df_bio['Bio Impudence']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_bio, y_bio, test_size=0.2, random_state=42)

# Train the RandomForestRegressor model
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error for Bio Impedance Prediction: {mse}')


      
# Example usage
dietary_preference = input("Enter your dietary preference (veg/non-veg): ")
weight = float(input("Enter your weight (kg): "))
height = float(input("Enter your height (m): "))
age = int(input("Enter your age: "))
gender = input("Enter your gender (Male/Female): ")
activity_level = input("Enter your activity level (sedentary/light/moderate/active/very_active): ")
weight_loss_plan = input("Enter your weight loss plan (Maintain weight/Lose Mild Weight/Lose Basic Weight/Lose Extreme Weight): ")



# Convert gender to a numerical value (1 for female, 0 for male, or adjust accordingly based on model)
gender_value = 1 if gender.lower() == 'female' else 0

# Prepare input for model prediction (same as what the model was trained on)
user_data = [[age, height, weight, gender_value]]  # Age, Height (m), Weight (kg), Gender (1 for female, 0 for male)

# Example prediction for bioimpedance using the model
predicted_bioimpedance = model.predict(user_data)
print(f'Predicted Bio Impedance: {predicted_bioimpedance[0]} Ohms')

get_meal_plan(dietary_preference, weight, height, age, gender, activity_level, weight_loss_plan, 0)

Mean Squared Error for Bio Impedance Prediction: 30292.09265438934
Enter your dietary preference (veg/non-veg): VEG
Enter your weight (kg): 46
Enter your height (m): 156
Enter your age: 24
Enter your gender (Male/Female): FEMALE
Enter your activity level (sedentary/light/moderate/active/very_active): LIGHT
Enter your weight loss plan (Maintain weight/Lose Mild Weight/Lose Basic Weight/Lose Extreme Weight): Lose Mild Weight
Predicted Bio Impedance: 544.0274999999999 Ohms
BMI: 18.90 (Normal weight)
BMR: 1154.00 kcal/day
TDEE (Total Daily Energy Expenditure): 1154.00 kcal/day
Calorie Requirement for Lose Mild Weight Plan: 904.00 kcal/day

Breakfast: 
- Uttapam
- Strawberries
- Coffee
- Chai
- Laal Chai
Total Breakfast Calories: 224

Lunch: 
- White Rice
Total Lunch Calories: 360

Dinner: 
- Honey
Total Dinner Calories: 304

Total Calories for the Day: 888, please add light snacks to your meals to meet your target Calorie requirements


(['Uttapam', 'Strawberries', 'Coffee', 'Chai', 'Laal Chai'],
 ['White Rice'],
 ['Honey'])